## LCM Function

In [1]:
import json
import random
import constraint
from techniques.PaDMTP import padmtp_algo
from techniques.RandomTesting import random_testing
from testcodes.lcm import compute_lcm as lcm
from techniques.AdaptiveRandomTesting import adaptive_random_testing

### PaDMTP

In [2]:
problem = constraint.Problem()

problem.addVariable('x', range(1, 16))
problem.addVariable('y', range(1, 16))

problem.addConstraint(lcm, ['x', 'y'])

solutions = problem.getSolutions()

length = len(solutions)

In [3]:
# Cleaning Dataset where values of 'x' or 'y' are 0(zero)

cleaned_solutions = []

for index, solution in enumerate(solutions):
    if solution['x'] == 0 or solution['y'] == 0:
        continue
    else:
        cleaned_solutions.append(solutions[index])
        
lcm_length = len(cleaned_solutions)

print("\nPossible Solutions Dataset for LCM Code: \n(x,y) ∈ {", end="")
for index, solution in enumerate(cleaned_solutions):
    if index == lcm_length - 1:
        print("({},{})".format(solution['x'], solution['y']), end="")
    else:
        print("({},{}),".format(solution['x'], solution['y']), end="")
print("}\n")


Possible Solutions Dataset for LCM Code: 
(x,y) ∈ {(15,15),(15,14),(15,13),(15,12),(15,11),(15,10),(15,9),(15,8),(15,7),(15,6),(15,5),(15,4),(15,3),(15,2),(15,1),(14,15),(14,14),(14,13),(14,12),(14,11),(14,10),(14,9),(14,8),(14,7),(14,6),(14,5),(14,4),(14,3),(14,2),(14,1),(13,15),(13,14),(13,13),(13,12),(13,11),(13,10),(13,9),(13,8),(13,7),(13,6),(13,5),(13,4),(13,3),(13,2),(13,1),(12,15),(12,14),(12,13),(12,12),(12,11),(12,10),(12,9),(12,8),(12,7),(12,6),(12,5),(12,4),(12,3),(12,2),(12,1),(11,15),(11,14),(11,13),(11,12),(11,11),(11,10),(11,9),(11,8),(11,7),(11,6),(11,5),(11,4),(11,3),(11,2),(11,1),(10,15),(10,14),(10,13),(10,12),(10,11),(10,10),(10,9),(10,8),(10,7),(10,6),(10,5),(10,4),(10,3),(10,2),(10,1),(9,15),(9,14),(9,13),(9,12),(9,11),(9,10),(9,9),(9,8),(9,7),(9,6),(9,5),(9,4),(9,3),(9,2),(9,1),(8,15),(8,14),(8,13),(8,12),(8,11),(8,10),(8,9),(8,8),(8,7),(8,6),(8,5),(8,4),(8,3),(8,2),(8,1),(7,15),(7,14),(7,13),(7,12),(7,11),(7,10),(7,9),(7,8),(7,7),(7,6),(7,5),(7,4),(7,3),(7,2),

In [4]:
print(f"LCM - PaDMTP - Test Cases: {lcm_length}")

LCM - PaDMTP - Test Cases: 225


In [5]:
mutPyObj = 'mut.py --target testcodes.lcm --unit-test mutations.test_lcm_padmt -m'

padmtp_rslt = padmtp_algo(lcm, cleaned_solutions, mutPyObj)

# ====================================================================== #
Source Test Case Execution (STCE) of 10% Dataset:
# ====================================================================== #

STCE (Sum of Results): 1950 

# ============================================================ #
Metamorphic Relations
# ============================================================ #

# ======================================================= #
1: Additive (Add a Positive Constant)
# ======================================================= #

Additive Value: 3350 

Additive Value has Increased

# ======================================================= #
2: Inclusive (Add a New Element)
# ======================================================= #

Inclusive Value: 1950 

EPA & Inclusive Values are Equal

# ======================================================= #
3: Permutative (Permutate a Value)
# ======================================================= #

Permutative Value: 3265 

Permutativ

Permutative Value: 18855 

Permutative Value has Increased

# ======================================================= #
4: Multiplicative (Multiply by a Positive Constant)
# ======================================================= #

Multiplicative Value: 37992 

Multiplicative Value has Increased


+-----------------------------------------------------------------------------------+
|                        Results: Metamorphic Relations (MRs)                       |
+-----------------------+-------------------------+--------------------+------------+
| Metamorphic Relations |            r1           |         r2         | Our_Result |
+-----------------------+-------------------------+--------------------+------------+
|        Additive       | Add a positive constant | Increase or Remain |    True    |
|       Inclusive       |    Add a new Element    | Increase or Remain |    True    |
|      Permutative      |    Permutate a Value    |       Remain       |   False    |
|     Multip

# ======================================================= #
PaDMTP Result: Mutation Testing (MT)
# ======================================================= #

 Mutation score [20.22113 s]: 72.7%
   - all: 12
   - killed: 4 (33.3%)
   - survived: 3 (25.0%)
   - incompetent: 1 (8.3%)
   - timeout: 4 (33.3%)




# ====================================================================== #
Source Test Case Execution (STCE) of 80% Dataset:
# ====================================================================== #

STCE (Sum of Results): 9750 

# ============================================================ #
Metamorphic Relations
# ============================================================ #

# ======================================================= #
1: Additive (Add a Positive Constant)
# ======================================================= #

Additive Value: 24504 

Additive Value has Increased

# ======================================================= #
2: Inclusive (Add a New Element)


In [6]:
padmtp_rslt['PaDMTP_Overhead']

21.21558117866516

In [7]:
padmtp_rslt['PaDMTP_DataFrame']

,k Test Cases,True MRs,False MRs,MT Score
0,10%,3,1,72.7%
1,20%,3,1,72.7%
2,30%,3,1,72.7%
3,40%,3,1,72.7%
4,50%,3,1,72.7%
5,60%,3,1,72.7%
6,70%,3,1,72.7%
7,80%,3,1,72.7%
8,90%,3,1,72.7%
9,100%,3,1,72.7%


In [8]:
PaDMTP_MT_Scores = []

for index, row in padmtp_rslt['PaDMTP_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    PaDMTP_MT_Scores.append(val)

### Random Testing (RT)

In [9]:
# Generating random inputs
rt_test_cases = []
for i in range(lcm_length):
    x = random.randint(1, 101)
    y = random.randint(1, 101)
    
    rt_test_cases.append((x, y))

In [10]:
print(f"LCM - Random Testing - Test Cases: {len(rt_test_cases)}")

LCM - Random Testing - Test Cases: 225


In [11]:
# Saving RT Test Cases to JSON File

jsonString = json.dumps(rt_test_cases)
jsonFile = open("./temp/RT_Test_Cases.json", 'w')
jsonFile.write(jsonString)
jsonFile.close()

In [12]:
pytest_cmd = 'pytest techniques/rt_lcm.py'
mutpy_cmd = 'mut.py --target testcodes.lcm --unit-test mutations.test_lcm_rt -m'

rt_rslt = random_testing(pytest_cmd, mutpy_cmd, rt_test_cases)

# ======================================================== #
Random Testing (RT) of 10% Dataset:
# ======================================================== #

# ================================= #
RT Results
# ================================= #

Total Tests: 22
Passed Tests: 22
Failed Tests: 0


# ================================= #
RT Result: Mutation Testing (MT)
# ================================= #

 Mutation score [10.16623 s]: 72.7%
   - all: 12
   - killed: 6 (50.0%)
   - survived: 3 (25.0%)
   - incompetent: 1 (8.3%)
   - timeout: 2 (16.7%)




# ======================================================== #
Random Testing (RT) of 20% Dataset:
# ======================================================== #

# ================================= #
RT Results
# ================================= #

Total Tests: 45
Passed Tests: 45
Failed Tests: 0


# ================================= #
RT Result: Mutation Testing (MT)
# ================================= #

 Mutation score [10.14192 s]: 72

In [13]:
rt_rslt['RT_Overhead']

11.157557010650635

In [14]:
rt_rslt['RT_DataFrame']

,k Test Cases,Total Tests,Passed Tests,Failed Tests,MT Score
0,10%,22,22,0,72.7%
1,20%,45,45,0,72.7%
2,30%,68,68,0,72.7%
3,40%,90,90,0,72.7%
4,50%,112,112,0,72.7%
5,60%,135,135,0,72.7%
6,70%,158,158,0,72.7%
7,80%,180,180,0,72.7%
8,90%,202,202,0,72.7%
9,100%,225,225,0,72.7%


In [15]:
RT_MT_Scores = []

for index, row in rt_rslt['RT_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    RT_MT_Scores.append(val)

### Adaptive Random Testing (ART)

In [16]:
# Generating random inputs
art_test_cases = []
for i in range(lcm_length):
    x = random.randint(1, 101)
    y = random.randint(1, 101)
    
    art_test_cases.append([x, y])

In [17]:
print(f"LCM - Adaptive Random Testing - Test Cases: {len(art_test_cases)}")

LCM - Adaptive Random Testing - Test Cases: 225


In [18]:
mutpy_cmd = 'mut.py --target testcodes.lcm --unit-test mutations.test_lcm_art -m'

art_rslt = adaptive_random_testing(lcm, mutpy_cmd, art_test_cases, 'LCM')

# ======================================================== #
Adaptive Random Testing (ART) of 10% Dataset:
# ======================================================== #

# ================================= #
RT Results
# ================================= #

Total Tests: 22
Passed Tests: 22
Failed Tests: 0


# ================================= #
RT Result: Mutation Testing (MT)
# ================================= #

 Mutation score [10.15033 s]: 72.7%
   - all: 12
   - killed: 6 (50.0%)
   - survived: 3 (25.0%)
   - incompetent: 1 (8.3%)
   - timeout: 2 (16.7%)




# ======================================================== #
Adaptive Random Testing (ART) of 20% Dataset:
# ======================================================== #

# ================================= #
RT Results
# ================================= #

Total Tests: 45
Passed Tests: 45
Failed Tests: 0


# ================================= #
RT Result: Mutation Testing (MT)
# ================================= #

 Mutation sc

In [19]:
art_rslt['ART_Overhead']

10.508551597595215

In [20]:
art_rslt['ART_DataFrame']

,k Test Cases,Total Tests,Passed Tests,Failed Tests,MT Score
0,10%,22,22,0,72.7%
1,20%,45,45,0,72.7%
2,30%,68,68,0,72.7%
3,40%,90,90,0,72.7%
4,50%,112,112,0,72.7%
5,60%,135,135,0,72.7%
6,70%,158,158,0,72.7%
7,80%,180,180,0,72.7%
8,90%,202,202,0,72.7%
9,100%,225,225,0,72.7%


In [21]:
ART_MT_Scores = []

for index, row in art_rslt['ART_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    ART_MT_Scores.append(val)

### Pairwise T-Test

In [22]:
import scipy.stats as stats
import numpy as np
from tabulate import tabulate

# Define the data
PaDMTP = np.array(PaDMTP_MT_Scores) 
RT = np.array(RT_MT_Scores)
ART = np.array(ART_MT_Scores)

# Calculate the mean
mean_padmtp = np.mean(PaDMTP)
mean_rt = np.mean(RT)
mean_art = np.mean(ART)

# Calculate the variance
var_padmtp = np.var(PaDMTP, ddof=1)
var_rt = np.var(RT, ddof=1)
var_art = np.var(ART, ddof=1)

# Get the number of observations
n_padmtp = len(PaDMTP)
n_rt = len(RT)
n_art = len(ART)

# Calculate the degrees of freedom
df_padmtp = n_padmtp - 1
df_rt = n_rt - 1
df_art = n_art - 1

# Perform t-tests
t_stat_rt, _ = stats.ttest_1samp(RT, popmean=mean_padmtp)
t_stat_art, _ = stats.ttest_1samp(ART, popmean=mean_padmtp)

# Create tables
table1 = [
    ["Test", "Mean", "Variance", "Observations", "df", "t Stat"],
    ["PaDMTP vs RT", mean_rt, var_rt, n_rt, df_rt, t_stat_rt],
    ["PaDMTP vs ART", mean_art, var_art, n_art, df_art, t_stat_art]
]

print("<==== LCM Code ====>")

# Print tables
print("Table 1:")
print(tabulate(table1, headers="firstrow", tablefmt="grid"))
# Perform one-sample t-tests
t_stat_rt, p_value_rt = stats.ttest_1samp(RT, popmean=np.mean(PaDMTP))
t_stat_art, p_value_art = stats.ttest_1samp(ART, popmean=np.mean(PaDMTP))

# Set the significance level (alpha)
alpha = 0.05

# Calculate the one-tail p-values and critical values
p_one_tail_rt = stats.t.cdf(t_stat_rt, df=len(RT)-1)
p_one_tail_art = stats.t.cdf(t_stat_art, df=len(ART)-1)
t_crit_one_tail_rt = stats.t.ppf(1-alpha, df=len(RT)-1)
t_crit_one_tail_art = stats.t.ppf(1-alpha, df=len(ART)-1)

# Calculate the two-tail p-values and critical values
p_two_tail_rt = 2 * (1 - stats.t.cdf(abs(t_stat_rt), df=len(RT)-1))
p_two_tail_art = 2 * (1 - stats.t.cdf(abs(t_stat_art), df=len(ART)-1))
t_crit_two_tail_rt = stats.t.ppf(1-alpha/2, df=len(RT)-1)
t_crit_two_tail_art = stats.t.ppf(1-alpha/2, df=len(ART)-1)
print("\n\n")
# Create table
table2 = [
    ["Test", "P(T<=t) one-tail", "t Critical one-tail", "P(T<=t) two-tail", "t Critical two-tail"],
    ["PaDMTP vs RT", p_one_tail_rt, t_crit_one_tail_rt, p_two_tail_rt, t_crit_two_tail_rt],
    ["PaDMTP vs ART", p_one_tail_art, t_crit_one_tail_art, p_two_tail_art, t_crit_two_tail_art]
]

# Print table
print("Table 2:")
print(tabulate(table2, headers="firstrow", tablefmt="grid"))
print("\n\n")
# Perform one-sample t-tests
t_stat_rt, p_value_rt = stats.ttest_1samp(RT, popmean=0.727)
t_stat_art, p_value_art = stats.ttest_1samp(ART, popmean=0.727)
print("Table 3:")
# Print the results
print("RT vs PaDMTP:")
print("t Statistic:", t_stat_rt)
print("p-value:", p_value_rt)
print("\nART vs PaDMTP:")
print("t Statistic:", t_stat_art)
print("p-value:", p_value_art)

<==== LCM Code ====>
Table 1:
+---------------+--------+-------------+----------------+------+----------+
| Test          |   Mean |    Variance |   Observations |   df |   t Stat |
+===============+========+=============+================+======+==========+
| PaDMTP vs RT  |  0.727 | 1.36955e-32 |             10 |    9 |        0 |
+---------------+--------+-------------+----------------+------+----------+
| PaDMTP vs ART |  0.727 | 1.36955e-32 |             10 |    9 |        0 |
+---------------+--------+-------------+----------------+------+----------+



Table 2:
+---------------+--------------------+-----------------------+--------------------+-----------------------+
| Test          |   P(T<=t) one-tail |   t Critical one-tail |   P(T<=t) two-tail |   t Critical two-tail |
+===============+====================+=======================+====================+=======================+
| PaDMTP vs RT  |                0.5 |               1.83311 |                  1 |               2.26

C:\Python311\Lib\site-packages\scipy\stats\_axis_nan_policy.py:502: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
